# 3.10

In [12]:
from torch import nn
import torch
import numpy as np

In [2]:
paris_coords = np.array([
    [48.8575, 2.3514], #Center of Paris
    [48.8584, 2.2945], # Eiffel Tower
    [48.8530, 2.3499], #Notre Dame
    [48.8606,  2.3376], #Louvre
    [48.8606, 2.3522]  #Centre Pompidou
])

madrid_coords = np.array([
    [40.4167, -3.7033],   # Center of Madrid
    [40.4153, -3.6835],   # Retiro Park 
    [40.4180, -3.7143],   # Royal Palace 
    [40.4138, -3.6921],   # Prado Museum 
    [40.4169, -3.7033]   # Puerta del Sol 
])

In [17]:
X = np.vstack([paris_coords, madrid_coords])
y = np.array([0, 0, 0, 0, 0,  #  Madrid labels (0)
              1, 1, 1, 1, 1,  #  Paris labels (1)
])

shuffle_idx = np.random.permutation(len(y))
X = X[shuffle_idx, 1:2]
y = y[shuffle_idx]

In [18]:
class MyNetwork(nn.Module):
    def __init__(self, input: int, output: int):
        super(MyNetwork, self).__init__()
        self.l1 = nn.Linear(input, output)
    
    def forward(self, x):

        return self.l1(x)


In [19]:
num_epoch = 100
lr = 0.005
model = MyNetwork(1, 2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr)

for i in range(num_epoch):

    optimizer.zero_grad()
    out = model(torch.tensor(X[i%len(y)]).float())
    loss = criterion(out, torch.tensor(y[i%len(y)]))

    loss.backward()
    optimizer.step()

    if (i + 1) % 10 == 0:
        with torch.no_grad():
            logits=model(torch.tensor(X, dtype=torch.float)) 
            accuracy=(torch.argmax(logits, dim=1)==torch.tensor(y)).sum().item()/len(y)
        print(f"Step {i+1}/{num_epoch}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")

Step 10/100, Loss: 3.5408, Accuracy: 0.0000
Step 20/100, Loss: 2.8838, Accuracy: 0.0000
Step 30/100, Loss: 2.2801, Accuracy: 0.0000
Step 40/100, Loss: 1.7659, Accuracy: 0.0000
Step 50/100, Loss: 1.3670, Accuracy: 0.0000
Step 60/100, Loss: 1.0771, Accuracy: 0.5000
Step 70/100, Loss: 0.8698, Accuracy: 0.5000
Step 80/100, Loss: 0.7195, Accuracy: 0.5000
Step 90/100, Loss: 0.6078, Accuracy: 1.0000
Step 100/100, Loss: 0.5226, Accuracy: 1.0000


Single Nueron GD

In [21]:
X = np.array([1, 2, 3, 4])
y = np.array([3,5,7,9])

shuffle_idx = np.random.permutation(len(y))

X = X[shuffle_idx]
y = y[shuffle_idx]

In [22]:
print(X)
print(y)

[4 1 2 3]
[9 3 5 7]


In [34]:
# RMS loss

def loss(model_out, gt):

    return (model_out-gt)**2

def m_grad(model_out, input, gt):

    return -2*(model_out-gt)*input

def b_grad(model_out, gt):

    return -2*(model_out-gt)

class Model:
    def __init__(self):
        self.m = 1
        self.b = 0
    
    def forward(self, input):

        return self.m*input + self.b

    def update(self, m_update, b_update):

        self.m += m_update
        self.b += b_update
    
    def get_model(self):

        print(f"Slope: {self.m} Intercept {self.b}")


def train(num_steps, lr):

    model = Model()

    for i in range(num_steps):

        index = i%len(y)
        input = X[index]
        gt = y[index]

        model_out = model.forward(input)
        loss_val = loss(model_out, gt)

        if i % 50 == 0:
            print(f"Epoch {i} Loss Val {loss_val}")

        m_update = m_grad(model_out, input, gt)
        b_update = b_grad(model_out, gt)

        model.update(m_update*lr, b_update*lr)
    
    model.get_model()




In [36]:
num_steps = 500
lr = 0.05

train(num_steps, lr)

Epoch 0 Loss Val 25
Epoch 50 Loss Val 0.019342487270405023
Epoch 100 Loss Val 0.002125427272660338
Epoch 150 Loss Val 0.00047381381958040497
Epoch 200 Loss Val 5.2064494096390025e-05
Epoch 250 Loss Val 1.1606549482844725e-05
Epoch 300 Loss Val 1.2753725240935499e-06
Epoch 350 Loss Val 2.8431418698846937e-07
Epoch 400 Loss Val 3.1241541927110554e-08
Epoch 450 Loss Val 6.964564019812605e-09
Slope: 2.000027663933508 Intercept 0.9999170081994775
